In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
import pandas as pd
import transformers
import torch
from transformers.utils import is_torch_available
import os
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from torch.utils.data import Dataset
from transformers import AutoModel, AutoTokenizer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForMaskedLM
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [ ]:
datasets_smiles = {
    "E. Coh. (Monomer)": (df_E_coh, "E_coh_bin"),
    "Tg (Monomer)": (df_T_g, "T_g_bin"),
    "R_gyr (Monomer)": (df_R_gyr, "R_gyr_bin"),
    "Densities (Monomer)": (df_Densities, "Densities_bin"),
    "Visc. Density": (df_dv, "viscosity_2bins_5050"),
    "BBBP": (df_bbbp, "p_np"),
    "Synthesability": (df_synth, "labels"),
    "Cycloadd. Energy": (df_cycloadd, "E_bin"),
    "Free Energy Cleavage": (df_catcleave, "deltaG4_realbin-27_-39"),
    "Melting Temperature of Molecules": (df_mp, "mp_bin"),
    "BACE": (df_bace, "label"),
    "Clintox FDA": (df_clintox_fda, "label"),
    "Clintox CT": (df_clintox_ct, "label"),
    "HIV": (df_hiv, "label"),
    "Hepatobiliary disorders": (dfs[0], "label"),
    "Endocrine disorders": (dfs[1], "label"),
    "Metabolism and nutrition disorders": (dfs[2], "label"),
    "Musculoskeletal and connective tissue disorders": (dfs[3], "label"),
    "Eye disorders": (dfs[4], "label"),
    "Vascular disorders": (dfs[5], "label"),
    "Gastrointestinal disorders": (dfs[6], "label"),
    "Skin and subcutaneous tissue disorders": (dfs[7], "label"),
    "General disorders and administration site conditions": (dfs[8], "label"),
    "Infections and infestations": (dfs[9], "label"),
    "Investigations": (dfs[10], "label"),
    "Renal and urinary disorders": (dfs[11], "label"),
    "Reproductive system and breast disorders": (dfs[12], "label"),
    "Respiratory, thoracic and mediastinal disorders": (dfs[13], "label"),
    "Nervous system disorders": (dfs[14], "label"),
    "Cardiac disorders": (dfs[15], "label"),
    "Blood and lymphatic system disorders": (dfs[16], "label"),
    "Psychiatric disorders": (dfs[17], "label"),
    "Neoplasms benign, malignant and unspecified (incl cysts and polyps)": (dfs[18], "label"),
    "Injury, poisoning and procedural complications": (dfs[19], "label"),
    "Ear and labyrinth disorders": (dfs[20], "label"),
    "Congenital, familial and genetic disorders": (dfs[21], "label"),
    "Immune system disorders": (dfs[22], "label"),
    "Pregnancy, puerperium and perinatal conditions": (dfs[23], "label"),
    "Social circumstances": (dfs[24], "label"),
    "Surgical and medical procedures": (dfs[25], "label"),
    "Product issues": (dfs[26], "label")

}

In [ ]:
model_dirs = [
    "bert-base-uncased",
    "DeepChem/ChemBERTa-77M-MLM",
    "DeepChem/ChemBERTa-77M-MTR",
    "ibm-research/MoLFormer-XL-both-10pct",
    "allenai/scibert_scivocab_uncased",
    "allenai/scibert_scivocab_cased",
    "answerdotai/ModernBERT-base",
    # "/home/david/modernbert_chemistry/fineweb/fine-web-modernbert-base-8192-multi-tok-1n4g-no-tags",
    # "/home/david/modernbert_chemistry/fineweb/fine-web-modernbert-base-8192-multi-tok-new-procedure-500k-notags",
    # "/home/david/modernbert_chemistry/fineweb/fine-web-modernbert-base-8192-multi-tok-new-procedure-1M5-notags",
    # "/home/david/modernbert_chemistry/fineweb/fine-web-modernbert-base-8192-multi-tok-new-procedure-10-epochs-notags",
    # "/home/david/modernbert_chemistry/fineweb/fine-web-modernbert-base-8192-multi-tok-new-procedure-20-epochs-notags",
    # "/home/david/modernbert_chemistry/fineweb/fine-web-modernbert-base-8192-multi-tok-new-procedure-30-epochs-notags",
    # "/home/david/modernbert_chemistry/fineweb/fine-web-modernbert-base-8192-multi-tok-new-procedure-40-epochs-notags",
    # "/home/david/modernbert_chemistry/fineweb/fine-web-modernbert-base-8192-multi-tok-new-procedure-50-epochs-notags",
    "/home/david/modernbert_chemistry/fineweb/fine-web-modernbert-base-8192-multi-tok-new-procedure-60-epochs-notags",

]

model_titles = [
    "bert-base-uncased",
    "ChemBERTa-MLM",
    "ChemBERTa-MTR",
    "MoLFormer",
    "SciBERT_uncased",
    "SciBERT_cased",
    "ModernBERT",
    # "ModernBERT base 500k",
    # "ModernBERT base Procedures 500k",
    # "ModernBERT base Procedures 1M5",
    # "ModernBERT base Procedures 1M5 10 epochs",
    # "ModernBERT base Procedures 1M5 20 epochs",
    # "ModernBERT base Procedures 1M5 30 epochs"
    # "ModernBERT base Procedures 1M5 40 epochs",
    # "ModernBERT base Procedures 1M5 50 epochs",
    "CheModernBERT"

]

from sklearn.metrics import f1_score

all_results = {}

n_experiments = 3

for model_dir in model_dirs:
    model_name = model_dir.split("/")[-1] 
    print(f"\nProcessing model: {model_name}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModelForMaskedLM.from_pretrained(model_dir)
    model.to(device)
    model.eval()
    
    def get_cls_embeddings(texts, batch_size=32):
        all_embeddings = []
        model.eval()
        with torch.no_grad():
            for i in range(0, len(texts), batch_size):
                batch_texts = [str(text) for text in texts[i:i+batch_size]]
                inputs = tokenizer(batch_texts, padding="longest", truncation=True,
                                return_tensors="pt", max_length=512)
                inputs = {k: v.to(device) for k, v in inputs.items()}
                outputs = model(**inputs, output_hidden_states=True)
                cls_embeddings = outputs.hidden_states[-1][:, 0, :]
                all_embeddings.append(cls_embeddings.cpu())
        return torch.cat(all_embeddings, dim=0).numpy()

    
    model_results = {}
    
    for task_name, (df_task, label_col) in datasets_smiles.items():
        print(f"  Processing task: {task_name}")

        smiles_list = df_task.iloc[:,0].tolist()
        cls_embeds = get_cls_embeddings(smiles_list)
        df_task["cls_embedding"] = [emb for emb in cls_embeds]
    
        X = np.stack(df_task["cls_embedding"].tolist())

        y = df_task["label"].values

        print(y.dtype)
        print(np.unique(y))
        y = y.astype(int)
        
        accuracies = []
        for i in range(n_experiments):
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=42 + i
            )
            rf = RandomForestClassifier(n_estimators=100, random_state=42 + i, n_jobs=-1)
            rf.fit(X_train, y_train)
            y_pred = rf.predict(X_test)
            acc = f1_score(y_test, y_pred)
            accuracies.append(acc)
        model_results[task_name] = accuracies
        mean_acc = np.mean(accuracies)
        std_acc = np.std(accuracies)
        print(f"    {task_name}: Mean Accuracy = {mean_acc:.4f}, Std Dev = {std_acc:.4f}")
    
    all_results[model_name] = model_results
